In [160]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.vq as scv
import pickle
import time

# os.system("cd $CAFFE_ROOT")
caffe_root = os.environ["CAFFE_ROOT"]
os.chdir(caffe_root)
print caffe_root
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_mode_gpu()
caffe.set_device(2)
option = 'lenet5'
if option == 'lenet5':
#     prototxt = '3_prototxt_solver/lenet5/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/lenet5/lenet5.caffemodel'
    solver_proto = '3_prototxt_solver/lenet5/lenet_solver_finetune.prototxt'
    iters = 100
    dir_t = '2_results/kmeans/lenet5/'
elif option == 'alexnet':
    prototxt = '3_prototxt_solver/L2/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/alexnet/alexnet9x.caffemodel'  
    solver_proto = '3_prototxt_solver/L2/finetune_solver.prototxt'
    iters = 1000
    dir_t = '2_results/kmeans/alexnet/'
elif option == 'vgg':
    prototxt = '3_prototxt_solver/vgg16/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/vgg16/vgg16_12x.caffemodel'  
    iters = 1000
    dir_t = '2_results/kmeans/vgg16/'

log = dir_t + 'log_accu'
print option

/home/songhan/pruning/
lenet5


In [161]:
solver = caffe.SGDSolver(solver_proto)
solver.net.copy_from(caffemodel)
net = solver.net

layers = ["conv1", "conv2", "ip1", "ip2"]
num_c = [8, 8, 8, 8]

# layers = ["ip2"]
# num_c =[4]
print "layers TBD: ", layers
print "num_c = ", num_c

layers TBD:  ['conv1', 'conv2', 'ip1', 'ip2']
num_c =  [8, 8, 8, 8]


In [162]:
print "==============1 Perform K-means============="
codebook = {}
for idx, layer in enumerate(layers):
    print "Eval layer:", layer
    W = net.params[layer][0].data.flatten()
    W = W[np.where(W != 0)]
    std = np.std(W)
    initial_uni = np.linspace(-4 * std, 4 * std, num_c[idx]-1)
    codebook[layer],_= scv.kmeans(W, initial_uni)    
    codebook[layer] = np.append(0.0, codebook[layer])
    print "codebook:", codebook[layer]
    print "codebook size:", len(codebook[layer])

==============1 Perform K-means=============
Eval layer: conv1
codebook: [ 0.         -0.29499978 -0.03177995  0.25980002  0.51812828]
codebook size: 5
Eval layer: conv2
codebook: [ 0.         -0.14003672 -0.08655009 -0.04095863  0.05826917  0.12395576
  0.21972357]
codebook size: 7
Eval layer: ip1
codebook: [ 0.         -0.07194122 -0.04054011 -0.01950156  0.01798702  0.03304647
  0.05246379  0.08222321]
codebook size: 8
Eval layer: ip2
codebook: [ 0.         -0.25988275 -0.17693673 -0.10709237  0.13464746  0.23517904]
codebook size: 6


In [163]:
def stochasitc_quantize(W, codebook):
#     print "codebook", codebook
#     print "W is:   ", W
    codebook=np.append(codebook,[np.inf])
    code0, dist0 = scv.vq(W, codebook)

    code1=[]
    dist1=[]
    for i in xrange(len(W)):
        d1=abs(W[i]-codebook[code0[i]-1])
        d2=abs(W[i]-codebook[code0[i]+1])
        if (d1>d2):
            code1.append(code0[i]+1)
            dist1.append(d2)
        else:
            code1.append(code0[i]-1)
            dist1.append(d1)

    r = np.random.uniform(low=0.0, high=1.0, size=(len(W)))

    sum_dist=dist0+dist1
    x=dist0/sum_dist

    mask0 = r>x
    mask1 = -mask0

    final_code = mask0*code0 + mask1*code1
    final_dist = mask0*dist0 + mask1*dist1
#     print "code0 is:", code0
#     print "dist0 is:", dist0
#     print "code1 is:", code1
#     print "dist1 is:", dist1
#     print dist0, r
#     print mask0
#     print mask1
    return final_code, final_dist
    

# for test:
codebook_ = np.array([0, 10, 20, 30, 40])
W_= np.array([12, 18, 24, 36])
final_code, final_dist = stochasitc_quantize( W_, codebook_)
print final_code, final_dist, codebook_
    

[1 1 3 3] [ 2.  8.  6.  6.] [ 0 10 20 30 40]


In [164]:
def quantize(net, layers, use_stochastic=False):
    print "================2 Perform quantization=============="
    codeDict={}
    maskCode={}
    for layer in layers:
#         print "Quantize layer:", layer
        W = net.params[layer][0].data
        if use_stochastic:
            codes, dist = stochasitc_quantize(W.flatten(), codebook[layer])        
        else:
            codes, dist = scv.vq(W.flatten(), codebook[layer])

        W_q = np.reshape(codebook[layer][codes], W.shape)
        net.params[layer][0].data[...] = W_q

        maskCode[layer] = np.reshape(codes, W.shape)
        codeBookSize = len(codebook[layer])    
#         print "W_q.shape=", W_q.shape        
#         print "codebook length=", codeBookSize
#         print "maskcode:", maskCode[layer].flatten().shape
        a = maskCode[layer].flatten()
        b = xrange(len(a))
        codeDict[layer]={}
        for i in xrange(len(a)):
            codeDict[layer].setdefault(a[i], []).append(b[i])

    return codeDict, maskCode

codeDict, maskCode = quantize(net, layers, False)

================2 Perform quantization==============


In [165]:
print "================3 Perform fintuning=============="
# print codebook
learning_rate=1e-3
decay_rate = 0.99 
momentum=0.9
update='rmsprop'
use_stochastic=False


import time
start_time=time.time()
step_cache={}
for i in xrange(10000):
    solver.step(1)
    for layer in layers:
        if not layer in step_cache: 
            step_cache[layer]={}
        diff=net.params[layer][0].diff.flatten()
#         W1 =  net.params[layer][0].data
        codeBookSize=len(codebook[layer])
        for code in xrange(codeBookSize):
            if code==0: continue;
            indexes = codeDict[layer][code]
            diff_ave=np.sum(diff[indexes])/len(indexes)

            if update == 'sgd':
                dx = -learning_rate * diff_ave
            elif update == 'momentum':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                dx = momentum * step_cache[layer][code] - learning_rate * diff_ave
                step_cache[layer][code] = dx                
            elif update == 'rmsprop':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                step_cache[layer][code] =  decay_rate * step_cache[layer][code] + (1.0 - decay_rate) * diff_ave ** 2
                dx = -(learning_rate * diff_ave) / np.sqrt(step_cache[layer][code] + 1e-8)
            elif update == 'adagrad':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                step_cache[layer][code] +=  diff_ave ** 2
                dx = -(learning_rate * diff_ave) / np.sqrt(step_cache[layer][code] + 1e-8)
            
            codebook[layer][code] += dx
        W2 = codebook[layer][maskCode[layer]]

#         if lr==0:
#             assert ((W1==W2).all())
        
        net.params[layer][0].data[...]=W2

    if (i+1)%500==0:
        print "iteration:", i, "requantize..."
        if use_stochastic:
            codeDict, maskCode = quantize(net, layers, True)
    if (i+1)%3000==0:
        learning_rate=learning_rate/10
        print "learning rate = ", learning_rate
    

print "time elapsed: ", time.time()-start_time 



================3 Perform fintuning==============
iteration: 499 requantize...


KeyboardInterrupt: 

In [ ]:
print net.params['ip2'][0].data[:10,0:3]
print codebook['ip2'][maskCode['ip2']][:10,0:3]


original net:
Test net output #0: accuracy = 0.992

after pruning to 8% parameters left and quantization all parameters to 3bits:
Test net output #0: accuracy = 0.9891

after retraining the codeBook:
     update == 'rmsprop':
     25 seconds
     lr = 1e-3 or 1e-2
Test net output #0: accuracy = 0.9914
accuracy loss = 0.06%


print "============ fine tune without codebook on Training Set ========="
print "batch size=", net.blobs['label'].data.shape
print "batch size=", solver.test_nets[0].blobs['label'].data.shape

import time
start_time = time.time()
for i in xrange(5000):
    solver.step(1)
        
print time.time()-start_time